In [50]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.chains.constitutional_ai.principles import PRINCIPLES
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.callbacks import get_openai_callback

import deeplake
import os
from dotenv import load_dotenv

load_dotenv()


True

In [51]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

my_activeloop_dataset_name = "spotify_review_dropped_text_clean_longsentences_20k"
dataset_path = f"./../deeplake/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings, num_workers=4, read_only=True)
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 50
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 25


Deep Lake Dataset in ./../deeplake/spotify_review_dropped_text_clean_longsentences_20k already exists, loading from the storage


In [52]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

Previous question:
{history}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question", "history"],
    template=template,
)

# Generation Only

In [50]:
principles = ['uo-evidence-2', 'uo-implications-2', 'uo-implications-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']
# principles = ['uo-evidence-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']

constitutional_principles = ConstitutionalChain.get_principles(principles)

llm = OpenAI(model="text-davinci-003", temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=llm_chain,
    constitutional_principles=constitutional_principles,
    llm=llm,
)

# istantiate the llm wrapper
# model = ChatOpenAI(model='gpt-3.5-turbo-1106')

# create the question-answering chain
# qa_chain = RetrievalQA.from_llm(model, retriever=retriever)

# ask a question to the chain
# qa_chain.run("When was Michael Jordan born?")


In [51]:
# user question
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(context=chunks_formatted, query=query)

# generate answer
answer = constitutional_chain.run(context=chunks_formatted, query=query)
print(answer)


Users dislike that they have to download a new application entirely, that the app update is unsatisfactory, that the user experience is bad, that there is a lack of app planning, and that the trial period is annoying and cumbersome. These issues need to be addressed in order to improve the user experience.


# Retrieval QA

In [101]:
# llm = OpenAI(model="text-davinci-003", temperature=0)
llm = ChatOpenAI(model='gpt-3.5-turbo-1106')
memory=ConversationBufferWindowMemory(
        memory_key="history",
        k=3,
        # input_key='input',
        # output_key='answer',
        return_messages=True)
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=retriever,
    memory=memory,
    chain_type_kwargs = {'prompt': prompt, 'verbose':True},
    verbose=True
)


In [102]:
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"

retrieval_qa.run(query=query)



> Entering new RetrievalQA chain...


ValueError: Missing some input keys: {'history'}

In [97]:
query = "What was the previous question?"
# query = "How to check disk usage in linux?"

retrieval_qa.run(query=query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
previous song ".

".

".

".

".

Question: What was the previous question?

Answer:


> Finished chain.

> Finished chain.

> Finished chain.


"I'm sorry, but there is no previous question provided in the context."

# Conversational QA

In [53]:
template = """You're a helpful assistant, please answer the following question correctly using only the information from the provided context. Do not invent stuff and simply say that you don't know to the answer because it is irrelevent with the provided context below:
{context}

This was our previous chat history:
{chat_history}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template,
)

In [54]:
# llm = OpenAI(model="text-davinci-003", temperature=0)
# llm = ChatOpenAI(model='gpt-3.5-turbo-1106')
# retrieval_qa = RetrievalQA.from_chain_type(
# 	llm=llm,
# 	chain_type="stuff",
# 	retriever=retriever,
# )

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.conversation.memory import ConversationSummaryMemory

# create GPT3 wrapper
compressor_llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create compressor for the retriever
compressor = LLMChainExtractor.from_llm(compressor_llm)
compression_retriever = ContextualCompressionRetriever(
	base_compressor=compressor,
	base_retriever=retriever
)

qa_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)
memory=ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=5,
        input_key='question',
        output_key='answer',
        return_messages=True)
conversation_qa = ConversationalRetrievalChain.from_llm(
	llm=qa_llm,
	chain_type="stuff",
    memory=memory,
    # memory=ConversationSummaryMemory(llm=),
	retriever=retriever,
    combine_docs_chain_kwargs={
        'prompt': prompt,
    },
    verbose=False
)

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain(query, return_only_outputs=True)
    return {
        'result': result,
        'token_count': cb.total_tokens
    }

In [55]:
query = "What do users dislikes about our application?"
# query = "How to check disk usage in linux?"
response = count_tokens(conversation_qa, {'question': query})
# response = conversation_qa(, return_only_outputs=True)

In [56]:
print(response)

{'result': {'answer': 'Users dislike the frequent need to download a new application entirely, the dissatisfaction with app updates, the cumbersome trial period, the lack of functionality compared to the desktop app, the frequent need to uninstall and reinstall the app, the requirement to delete and download a new app, the frequent crashes and slow response time, the app being buggy and temperamental, and the limitations of the free version.'}, 'token_count': 1220}


In [34]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What do users dislikes about our application?'),
  AIMessage(content='Users dislike the need to download a new application entirely, dissatisfaction with updates, the requirement to download offline files, a cumbersome trial period, and the installation process.')]}

In [48]:
query = "What do users likes about our application?"
# query = "How to check disk usage in linux?"

response = count_tokens(conversation_qa, {'question': query})

# response = conversation_qa({'question': query}, return_only_outputs=True)

In [49]:
print(response)

{'result': {'answer': 'Users like the convenience, ease of use, smart features, user-friendly interface, great features, fast downloads, and smooth UI of the application.'}, 'token_count': 552}


In [37]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='What do users dislikes about our application?'),
  AIMessage(content='Users dislike the need to download a new application entirely, dissatisfaction with updates, the requirement to download offline files, a cumbersome trial period, and the installation process.'),
  HumanMessage(content='What do users likes about our application?'),
  AIMessage(content='Users like the convenience, ease of use, smart features, user-friendly interface, great app features, fast downloads, and smooth user interface.')]}

In [38]:
query = "What was my first question?"
# query = "How to check disk usage in linux?"

conversation_qa.run({'question': query})

'What do users dislikes about our application?'

In [26]:
query = "Based on the user reviews, what do we need to do to improve our application?"
# query = "How to check disk usage in linux?"

conversation_qa.run({'question': query})

'Based on the user reviews, it seems that the application could benefit from improvements in searching, music diversity categorizations, and making changes and fixes. Additionally, users have expressed a need for the app to keep getting better.'

In [27]:
query = "What kind of app changes that we needed?"
# query = "How to check disk usage in linux?"

conversation_qa.run({'question': query})

'The app needs to make changes and fixes, upgrade soon, and be able to pay on two devices. Users also want the app to listen at home, work, and on road trips with cell service, and to change the UI often. Additionally, the app needs to be remade entirely and must be a must-have for music junkies and anyone who likes music.'

In [28]:
query = "How to check disk usage in linux?"

conversation_qa.run({'question': query})

"I'm sorry, I don't have that information based on the provided context."

# Conversation with Constitunional Principles

In [74]:
constitutional_template = """Given a text or paragraph that was generated from a response of other llm, your job is to apply transformation techniques such as summerization or grammar corrections. Only apply if needed and leave it as it is if you think it is readable enough for the reader.

{answer}
"""

constitutional_prompt = PromptTemplate(
    input_variables=["answer"],
    template=constitutional_template,
)

In [75]:
from langchain.chains import SequentialChain

principles = ['uo-evidence-2', 'uo-implications-2', 'uo-implications-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']
# principles = ['uo-evidence-3', 'uo-reasoning-3', 'uo-utility-3', 'uo-security-1']

constitutional_principles = ConstitutionalChain.get_principles(principles)

qa_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)
memory=ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=5,
        # input_key='input',
        output_key='answer',
        return_messages=True)

conversation_qa = ConversationalRetrievalChain.from_llm(
	llm=qa_llm,
	chain_type="stuff",
    memory=memory,
    # memory=ConversationSummaryMemory(llm=),
	retriever=retriever,
    combine_docs_chain_kwargs={
        'prompt': prompt,
    },
    verbose=False,
    # output_keys=["answer"],
)


constitunioal_llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
llm_chain = LLMChain(llm=constitunioal_llm, prompt=constitutional_prompt)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=llm_chain,
    constitutional_principles=constitutional_principles,
    llm=constitunioal_llm,
    # output_key="refined_answer",
)

#Overall_chain
overall_chain = SequentialChain(
    chains=[conversation_qa, constitutional_chain ],
    input_variables=["question"],
    # output_variables=["refined_answer"],
    verbose=True)

In [77]:
query = "What kind of app version that makes the application rating low?"
# query = "How to check disk usage in linux?"

overall_chain.run({'question': query})



> Entering new SequentialChain chain...

> Finished chain.


'\nSome users have requested for the app to be more user-friendly and to have the option to revert back to the old version. Additionally, they have also expressed a desire for the ability to delete multiple items simultaneously.'

# TEMPLATE LEGACY

In [ ]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

In [ ]:
# Search the database for a response based on the user's query
def search_db(user_input, db):
    print(user_input)
    retriever = db.as_retriever()
    retriever.search_kwargs['distance_metric'] = 'cos'
    retriever.search_kwargs['fetch_k'] = 100
    retriever.search_kwargs['maximal_marginal_relevance'] = True
    retriever.search_kwargs['k'] = 4
    model = ChatOpenAI(model_name='gpt-3.5-turbo')
    qa = RetrievalQA.from_llm(model, retriever=retriever, return_source_documents=True)
    return qa({'query': user_input})


In [ ]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

In [32]:

print(f"{len(PRINCIPLES)} available principles")
for item in PRINCIPLES.items():
    print(item)
    print("#############\n")
# list(PRINCIPLES.items())

54 available principles
('harmful1', ConstitutionalPrinciple(critique_request='Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.', revision_request='Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.', name='harmful1'))
#############

('harmful2', ConstitutionalPrinciple(critique_request='Explain ways in which the assistant’s preceding response may be harmful to either the human or others. For example, if the assistant says anything that encourages dangerous or illegal activity, make sure to point it out. Other examples of harm include racism, sexism, and other social biases.', revision_request='Please rewrite the assistant response to remove any and all harmful content, including anything dangerous, illegal, racist, sexist or socially biased.', name='harmful2'))
#############

('harmful3', ConstitutionalPrinciple(critique_r

In [ ]:
('uo-evidence-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response contains all relevant information, and does **not** contain irrelevant information.", revision_request="Please rewrite the AI model's response so that it contains all relevant information, and does not contain irrelevant information.", name='uo-evidence-3'))
#############

('uo-implications-2', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response lists appropriate suggestions for further actions or requests.", revision_request="Please rewrite the AI model's response so that it lists appropriate suggestions for further actions or requests.", name='uo-implications-2'))
#############

('uo-implications-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response indicates if no further actions or requests are required.", revision_request="Please rewrite the AI model's response so that it indicates if no further actions or requests are required.", name='uo-implications-3'))
#############

('uo-reasoning-3', ConstitutionalPrinciple(critique_request="Discuss whether reasoning in the AI model's response is structured (e.g. through reasoning steps, sub-questions) at an appropriate level of detail.", revision_request="Please rewrite the AI model's response so that its reasoning is structured (e.g. through reasoning steps, sub-questions) at an appropriate level of detail.", name='uo-reasoning-3'))
#############


('uo-utility-3', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response is well-formatted, e.g. free from syntactic or grammatical errors.", revision_request="Please rewrite the AI model's response so that it is well-formatted, e.g. free from syntactic or grammatical errors.", name='uo-utility-3'))
#############

('uo-security-1', ConstitutionalPrinciple(critique_request="Discuss whether the AI model's response appropriately handles incorrect, incomplete, ambiguous, or irrelevant requests or information.", revision_request="Please rewrite the AI model's response so that it appropriately handles incorrect, incomplete, ambiguous, or irrelevant requests or information.", name='uo-security-1'))
#############

